# Exploración de Datos Financieros y Macroeconómicos

## Objetivo del Notebook

Este notebook corresponde a la **fase de exploración y análisis descriptivo de datos** del proyecto predictivo de tipo de cambio USD/EUR. Utiliza información previamente recopilada y almacenada en la carpeta `data/`, y tiene como propósito:

- Validar la integridad y calidad de los datos
- Analizar patrones temporales relevantes
- Identificar correlaciones entre variables clave
- Establecer fundamentos para la fase de modelado

##  Datos utilizados

Los archivos `.csv` que se cargan en esta etapa provienen del notebook previo (`00_carga_datos.ipynb`) y han sido organizados para facilitar el análisis modular.

##  Estructura del Notebook

1. Importación de librerías
2. Carga de archivos desde carpeta `data/`
3. Análisis exploratorio individual por variable
4. Visualizaciones combinadas
5. Detección de outliers y patrones atípicos
6. Preparación para modelado

---

Este análisis servirá de base para construir escenarios predictivos fundamentados y estrategias de cobertura cambiaria adaptadas a contextos macroeconómicos reales.


## Origen y Procedencia de los Datos

Para garantizar la fiabilidad del modelo y su aplicabilidad en entornos reales, se han utilizado fuentes oficiales y reconocidas en el ámbito financiero y económico:

### 1. Tipo de Cambio USD/EUR (`USD_EUR.csv`)

- **Fuente**: [Yahoo Finance](https://finance.yahoo.com/quote/EURUSD=X)
- **Ticker**: `EURUSD=X`
- **Frecuencia**: Diaria
- **Variable**: Precio de cierre del dólar respecto al euro

###  2. Índice del Dólar DXY (`DXY_Index.csv`)

- **Fuente**: [Yahoo Finance](https://finance.yahoo.com/quote/DX-Y.NYB)
- **Ticker**: `DX-Y.NYB`
- **Frecuencia**: Diaria
- **Variable**: Cierre del índice del dólar frente a una cesta de monedas principales

### 3. Inflación en EE.UU. (`Inflacion_USA.csv`)

- **Fuente**: [FRED – Federal Reserve Economic Data](https://fred.stlouisfed.org/series/CPIAUCSL)
- **Serie**: `CPIAUCSL`
- **Frecuencia**: Mensual
- **Variable**: Índice de Precios al Consumidor para consumidores urbanos en EE.UU.

###  4. Tasa de Interés de la Reserva Federal (`Tasa_Interes_Fed.csv`)

- **Fuente**: [FRED – Federal Reserve Economic Data](https://fred.stlouisfed.org/series/FEDFUNDS)
- **Serie**: `FEDFUNDS`
- **Frecuencia**: Mensual
- **Variable**: Tasa de fondos federales efectiva

---

Todos los datos han sido recolectados entre 2010 y 2025 y normalizados para su integración en el modelo. Se encuentran en formato `.csv` dentro de la carpeta `data/`, listos para análisis y modelado en notebooks posteriores.


In [2]:
# Manipulación de datos
import pandas as pd
import numpy as np

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de gráficos
%matplotlib inline
sns.set(style='whitegrid', palette='muted', context='talk')

# Manejo de fechas
from datetime import datetime


In [3]:
import os
os.listdir()


['.git',
 '01_exploracion_datos.ipynb',
 'carga de datos de red a csv.ipynb',
 'DXY_Index.csv',
 'Inflacion_USA.csv',
 'logo grupo.JPG',
 'README.md',
 'requirements.txt',
 'Tasa_Interes_Fed.csv',
 'USD_EUR.csv']

In [6]:
# Carga sin indexación ni conversión de fechas
usd_eur_raw     = pd.read_csv('USD_EUR.csv')
dxy_raw         = pd.read_csv('DXY_Index.csv')
inflacion_raw   = pd.read_csv('Inflacion_USA.csv')
fed_rate_raw    = pd.read_csv('Tasa_Interes_Fed.csv')

# Vista rápida para confirmar estructura
print("USD_EUR.csv:\n", usd_eur_raw.head(), "\n")
print("DXY_Index.csv:\n", dxy_raw.head(), "\n")
print("Inflacion_USA.csv:\n", inflacion_raw.head(), "\n")
print("Tasa_Interes_Fed.csv:\n", fed_rate_raw.head(), "\n")


USD_EUR.csv:
         Price             USD/EUR
0      Ticker            EURUSD=X
1       Fecha                 NaN
2  2010-01-01  1.4389938116073608
3  2010-01-04  1.4423978328704834
4  2010-01-05  1.4365959167480469 

DXY_Index.csv:
         Price                DXY
0      Ticker           DX-Y.NYB
1       Fecha                NaN
2  2010-01-04  77.52999877929688
3  2010-01-05  77.62000274658203
4  2010-01-06  77.48999786376953 

Inflacion_USA.csv:
         Fecha  Inflación USA
0  1947-01-01          21.48
1  1947-02-01          21.62
2  1947-03-01          22.00
3  1947-04-01          22.00
4  1947-05-01          21.95 

Tasa_Interes_Fed.csv:
         Fecha  Tasa Fed
0  1954-07-01      0.80
1  1954-08-01      1.22
2  1954-09-01      1.07
3  1954-10-01      0.85
4  1954-11-01      0.83 



##  Inicio de la Fase de EDA, Limpieza y Gestión de Datos

Con los datos económicos ya recopilados y almacenados en formato `.csv`, comenzamos ahora la fase de **análisis exploratorio**, **depuración estructural** y **normalización** de las variables clave.

### Objetivos de esta etapa

- Examinar la estructura interna de cada dataset
- Detectar y corregir inconsistencias o formatos irregulares
- Establecer índices temporales homogéneos para todos los DataFrames
- Preparar los datos para visualización, modelado y simulación de escenarios

### Datasets en bruto incluidos

- Tipo de cambio USD/EUR
- Índice del dólar DXY
- Inflación mensual en EE.UU.
- Tasa de interés efectiva de la Reserva Federal

---

Este proceso garantiza una base sólida para las siguientes etapas del proyecto: desde la construcción del modelo predictivo hasta el diseño de la estrategia de cobertura cambiaria.


In [7]:
# Partimos del archivo original cargado sin procesar
usd_eur = usd_eur_raw.copy()

# Eliminamos las dos primeras filas no numéricas (ticker y cabecera repetida)
usd_eur = usd_eur.iloc[2:]

# Renombramos columnas para mayor claridad
usd_eur = usd_eur.rename(columns={'Price': 'Fecha', 'USD/EUR': 'USD_EUR'})

# Convertimos la columna 'Fecha' a datetime
usd_eur['Fecha'] = pd.to_datetime(usd_eur['Fecha'])

# Establecemos la fecha como índice
usd_eur = usd_eur.set_index('Fecha')

# Aseguramos que los valores sean numéricos
usd_eur = usd_eur.astype(float)

# Vista rápida para comprobar
usd_eur.head()


,USD_EUR
Fecha,
2010-01-01,1.438994
2010-01-04,1.442398
2010-01-05,1.436596
2010-01-06,1.440403
2010-01-07,1.431803


In [8]:
# Limpieza del archivo DXY_Index.csv
dxy = dxy_raw.copy()

# Eliminamos las dos primeras filas no numéricas
dxy = dxy.iloc[2:]

# Renombramos columnas
dxy = dxy.rename(columns={'Price': 'Fecha', 'DXY': 'DXY'})

# Convertimos 'Fecha' a formato datetime
dxy['Fecha'] = pd.to_datetime(dxy['Fecha'])

# Establecemos como índice temporal
dxy = dxy.set_index('Fecha')

# Convertimos a valores numéricos
dxy = dxy.astype(float)

# Vista rápida
dxy.head()


,DXY
Fecha,
2010-01-04,77.529999
2010-01-05,77.620003
2010-01-06,77.489998
2010-01-07,77.910004
2010-01-08,77.470001


In [9]:
inflacion = inflacion_raw.copy()
inflacion['Fecha'] = pd.to_datetime(inflacion['Fecha'])
inflacion = inflacion.set_index('Fecha')
inflacion = inflacion.astype(float)
inflacion.head()


,Inflación USA
Fecha,
1947-01-01,21.48
1947-02-01,21.62
1947-03-01,22.00
1947-04-01,22.00
1947-05-01,21.95


In [10]:
fed_rate = fed_rate_raw.copy()
fed_rate['Fecha'] = pd.to_datetime(fed_rate['Fecha'])
fed_rate = fed_rate.set_index('Fecha')
fed_rate = fed_rate.astype(float)
fed_rate.head()


,Tasa Fed
Fecha,
1954-07-01,0.80
1954-08-01,1.22
1954-09-01,1.07
1954-10-01,0.85
1954-11-01,0.83


##  Integración de Series Temporales Económicas

Con las cuatro variables clave ya depuradas y correctamente indexadas por fecha, procedemos a integrar todos los DataFrames en una estructura conjunta. Esto permitirá:

- Visualizar relaciones históricas entre indicadores cambiarios y macroeconómicos
- Detectar sincronicidades, rupturas o eventos comunes
- Preparar una base consolidada para el modelado predictivo

###  Series a unir

- USD/EUR — Tipo de cambio diario
- DXY — Índice del dólar diario
- Inflación USA — Índice mensual de precios al consumidor
- Tasa de interés Fed — Tasa mensual efectiva de fondos federales

La fusión se realizará mediante uniones por índice temporal (`Fecha`), conservando sólo aquellas fechas donde se dispone de datos para todas las series.


In [11]:
# Aseguramos que todos los índices estén bien nombrados
usd_eur.index.name = 'Fecha'
dxy.index.name = 'Fecha'
inflacion.index.name = 'Fecha'
fed_rate.index.name = 'Fecha'

# Unimos progresivamente por índice temporal
df_merged = usd_eur.join(dxy, how='outer') \
                   .join(inflacion, how='outer') \
                   .join(fed_rate, how='outer')

# Eliminamos fechas incompletas para asegurar consistencia analítica
df_merged = df_merged.dropna()

# Vista rápida del DataFrame final
df_merged.head()


,USD_EUR,DXY,Inflación USA,Tasa Fed
Fecha,,,,
2010-02-01,1.392699,79.239998,217.281,0.13
2010-03-01,1.356208,80.660004,217.353,0.16
2010-04-01,1.358400,80.779999,217.403,0.20
2010-06-01,1.223301,86.639999,217.199,0.18
2010-07-01,1.250750,84.720001,217.605,0.18


In [12]:
df_merged.to_csv('dataset_final_economico.csv')
print("✅ Dataset final guardado como 'dataset_final_economico.csv'")


✅ Dataset final guardado como 'dataset_final_economico.csv'


## Descripción de Variables – Dataset Final Económico

Este dataset consolidado agrupa las principales variables económicas utilizadas en el proyecto para modelar la evolución del tipo de cambio del dólar. Todas las series han sido limpiadas, alineadas temporalmente y tipadas correctamente.

###  USD_EUR

- **Nombre original:** USD/EUR
- **Fuente:** Yahoo Finance (`EURUSD=X`)
- **Frecuencia:** Diaria
- **Descripción:** Valor diario de cierre del tipo de cambio entre el dólar estadounidense (USD) y el euro (EUR).
- **Relevancia:** Indicador directo de la paridad cambiaria bilateral. Es la variable objetivo del modelo predictivo y refleja las fluctuaciones cambiarias observadas en el mercado.

---

### DXY

- **Nombre original:** DXY
- **Fuente:** Yahoo Finance (`DX-Y.NYB`)
- **Frecuencia:** Diaria
- **Descripción:** Índice ponderado que representa la fortaleza del dólar frente a una cesta de divisas extranjeras principales (EUR, JPY, GBP, CAD, SEK, CHF).
- **Relevancia:** Funciona como indicador agregado del valor del dólar y su comportamiento global. Útil para detectar tendencias generales y correlaciones con movimientos en el tipo de cambio bilateral USD/EUR.

---

### Inflación USA

- **Nombre original:** Inflación USA
- **Fuente:** FRED – St. Louis Federal Reserve (`CPIAUCSL`)
- **Frecuencia:** Mensual (interpolada a frecuencia diaria para alineación)
- **Descripción:** Índice de Precios al Consumidor (IPC) para consumidores urbanos en EE.UU., ajustado estacionalmente.
- **Relevancia:** Variable clave de análisis macroeconómico. La inflación impacta las decisiones de política monetaria de la Fed y, por tanto, influye indirectamente sobre el valor del dólar.

---

### Tasa Fed

- **Nombre original:** Tasa Fed
- **Fuente:** FRED (`FEDFUNDS`)
- **Frecuencia:** Mensual (interpolada a frecuencia diaria para alineación)
- **Descripción:** Tasa efectiva de fondos federales. Es el principal instrumento de la Reserva Federal para regular el acceso al crédito y la oferta monetaria.
- **Relevancia:** Afecta directamente la apreciación o depreciación del dólar. Es utilizada en el modelo como variable exógena que refleja la postura monetaria del banco central.

---

### Índice Temporal: Fecha

- **Nombre del índice:** Fecha
- **Tipo de dato:** `datetime` (formato estándar AAAA-MM-DD)
- **Descripción:** Fecha correspondiente a cada observación en el dataset. Funciona como columna índice, permitiendo operaciones temporales, resampling, agregación y visualización dinámica.

---

Cada una de estas variables fue seleccionada con base en su poder explicativo y su relación teórica con el comportamiento cambiario. El dataset está listo para alimentar modelos de series temporales, simulación de escenarios y estrategias de cobertura.
